# Few-Shot Prompting
https://platform.openai.com/docs/guides/prompt-engineering

In some cases, it's easier to show the model what you want rather than tell the model what you want.
<br><br>
One way to show the model what you want is with creating a few fake back-and-forth messages between user and assistant. This is called `few-shot prompting`. 
<br><br>
The opposite of few-shot prompting is `zero-shot prompting` (previous examples).
***

## Prerequisites

1. Make sure that `python3` is installed on your system.
1. Create and Activate a Virtual Environment: <br><br>
    `python3 -m venv venv` <br>
    `source venv/bin/activate` <br><br>
1. Create a `.env` file in the same directory as this script and add the following variables:<br><br>
     ```
     AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>
     AZURE_OPENAI_MODEL=<your_azure_openai_model>
     AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>
     AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>
     ```
***

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [9]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


***
## Import Modules

In [10]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.
from pprint import pprint       # The `pprint` library is used to pretty-print a dictionary

## Load environment variables from .env file

In [11]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create an instance of the AzureOpenAI client
- The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API.
- It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [12]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Set the behavior or personality of the assistant by providing fake conversations

In this example, we are expecting the assistant to respond in Hindi.

To achieve this, we will provide a series of examples in the `developer` message that simulate such conversation and instruct the model to answer based on the pattern of the conversation.

For elaborate developer and user messages, OpenAI recommends using a combination of Markdown formatting and XML tags to help the model understand logical boundaries of your prompt and context data.


In [13]:
llm_message = f"""
# Instruction
You answer based on the pattern of the conversation.

# Examples
<user_query id="example-1">Hi, how are you?</user_query>
<assistant_response id="example-1">Main accha hoon, aap kaise hain?</assistant_response>
<user_query id="example-2">I am fine, can you tell me something?</user_query>
<assistant_response id="example-2">Haan, bilkul! Aapko kya jaanana hai?</assistant_response>
"""
conversation=[{"role": "developer", "content": llm_message}]

## Call the Responses API to get the AI's response

In [14]:
def talk_ai(question):
    
    # --------------------------------------------------------------
    # Append user question to the conversation history
    # --------------------------------------------------------------
    conversation.append({"role": "user", "content": question})

    try:
        response = client.responses.create(
            model= AZURE_OPENAI_MODEL,
            input=conversation,
            temperature=0.7,
            max_output_tokens=1000
        )

        # --------------------------------------------------------------
        # Extract answer and print it
        # --------------------------------------------------------------
        answer = response.output_text
        print(f"Answer from AI = {answer}")
        print(f"input tokens = {response.usage.input_tokens}")
        print(f"output tokens = {response.usage.output_tokens}")
        print(f"total tokens = {response.usage.total_tokens}")

        # --------------------------------------------------------------
        # Append the assistant's response to the conversation history
        # --------------------------------------------------------------
        conversation.append({"role": "assistant", "content": answer})

    except Exception as e:
        print(f"Error getting answer from AI: {e}")

## Prompt user for question, get response from LLM

In [15]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
talk_ai(question)


Question: Hi! My name is Agni
Answer from AI = Namaste Agni! Aap se milkar khushi hui. Aap kaise hain?
input tokens = 125
output tokens = 21
total tokens = 146


## Ask again

In [16]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
talk_ai(question)

Question: How are you today?
Answer from AI = Main aaj bahut achha hoon, dhanyavaad! Aap ka din kaisa ja raha hai?
input tokens = 158
output tokens = 26
total tokens = 184
